## Contextual Bandit
There are states involoved in this kind of problem, however they aren't determined by the previous states or actions.

Multiple bandits instead of one, and each has its probability distribution for reward.

In [7]:
import tensorflow as tf
import numpy as np

import tensorflow.contrib.slim as slim

three four-armed bandits, three here coresponding to three different environment


In [64]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        #List out our bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal.
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self): # returns a scalar value as state (from three possible states)
        self.state = np.random.randint(0,len(self.bandits)) #Returns a random state for each episode.
        return self.state
        
    def pullArm(self,action):
        #Get a random number.
        bandit = self.bandits[self.state,action]
        result = np.random.rand(1) ## if np.random.randn(1) then the result become highly unstable 
        if result > bandit:
            #return a positive reward.
            return 1
        else:
            #return a negative reward.
            return -1

In [59]:
class agent():
    def __init__(self,lr,s_size,a_size):
        ## agent takes state and output action
        self.state_in = tf.placeholder(shape=[1],dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size) ## OH One_Hot representation of state
        output = slim.fully_connected(state_in_OH,a_size,biases_initializer=None, activation_fn = tf.nn.relu, weights_initializer = tf.ones_initializer())
        self.out = output
        self.output = tf.reshape(output,[-1]) # from dimension (1,4) become dimension(4,)
        self.chosen_action = tf.argmax(self.output,0)
        
        self.reward_holder = tf.placeholder(shape=[1],dtype = tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype = tf.int32)
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [65]:
tf.reset_default_graph()
cBandit = contextual_bandit()
myAgent = agent(lr=0.001,s_size=cBandit.num_bandits,a_size=cBandit.num_actions) #Load the agent.
weights = tf.trainable_variables()[0]



total_episodes = 10000
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions])
e=0.1

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit() #Get a state from the environment.
        
        #Choose either a random action or one from our network.
        #Exploration - Exploitation trade-off principle here
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action,feed_dict={myAgent.state_in:[s]})
        
        print "myagent out is"
        print myAgent.
        reward = cBandit.pullArm(action) #Get our reward for taking an action given a bandit.
        
        #Update the network.
        feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update,weights], feed_dict=feed_dict)
        
        #Update our running tally of scores.
        total_reward[s,action] += reward
        if i % 500 == 0:
            print "Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1))
        i+=1
for a in range(cBandit.num_bandits):
    print "The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising...."
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print "...and it was right!"
    else:
        print "...and it was wrong!"

Mean reward for each of the 3 bandits: [ 0.   -0.25  0.  ]
Mean reward for each of the 3 bandits: [ 33.75  42.    33.  ]
Mean reward for each of the 3 bandits: [ 74.5   77.5   64.25]
Mean reward for each of the 3 bandits: [ 109.    120.25  100.5 ]
Mean reward for each of the 3 bandits: [ 147.75  163.5   132.5 ]
Mean reward for each of the 3 bandits: [ 190.    204.    165.75]
Mean reward for each of the 3 bandits: [ 225.75  241.    204.  ]
Mean reward for each of the 3 bandits: [ 266.    277.25  237.5 ]
Mean reward for each of the 3 bandits: [ 302.25  314.    276.5 ]
Mean reward for each of the 3 bandits: [ 337.5   349.5   310.75]
Mean reward for each of the 3 bandits: [ 375.    380.5   351.25]
Mean reward for each of the 3 bandits: [ 412.75  415.5   390.  ]
Mean reward for each of the 3 bandits: [ 454.25  450.25  423.75]
Mean reward for each of the 3 bandits: [ 497.    484.25  461.  ]
Mean reward for each of the 3 bandits: [ 537.25  521.25  496.75]
Mean reward for each of the 3 bandits

In [71]:
with tf.Session() as sess:
    sess.run(init)
    print myAgent.out.eval()

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype int32 and shape [1]
	 [[Node: Placeholder = Placeholder[dtype=DT_INT32, shape=[1], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'Placeholder', defined at:
  File "/Users/yitongxu/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/yitongxu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/yitongxu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/yitongxu/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/yitongxu/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/yitongxu/anaconda2/lib/python2.7/site-packages/zmq/e